In [1]:
import pandas as pd
import uuid
#from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf

# Used in FL
import nest_asyncio
nest_asyncio.apply()
import collections
import tensorflow_federated as tff
import tensorflow as tf

np.random.seed(0)
import random

2023-04-28 20:59:34.236505: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-28 20:59:34.431639: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-28 20:59:34.432908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 20:59:39.671017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Testing TFF
print(tff.federated_computation(lambda: 'Hello, World!')())

2023-04-28 20:59:54.874025: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 20:59:54.875086: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
E0428 20:59:55.111553100     146 socket_utils_common_posix.cc:221] check for SO_REUSEPORT: {"created":"@1682729995.111523100","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":199,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
E0428 20:59:55.112206200     146 socket_utils_common_posix.cc:327] setsockopt(TCP_USER_TIMEOUT) Protocol not available


b'Hello, World!'


In [3]:
# Read the CSV files into separate dataframes
df_ey = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_younger.csv')
df_eo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_older.csv')
df_ly = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_younger.csv')
df_lo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_older.csv')

In [4]:
# Concatenate the dataframes into one dataframe with matching headers
df = pd.concat([df_ey, df_eo, df_ly, df_lo], ignore_index=True, sort=False)

In [5]:
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
0,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
1,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
2,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
3,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
4,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
870,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
871,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0
872,1,1,3,2,1,1,48.0,0.565451,13.000000,0.310525,191.393162,3.060358,1.0,NoDx,55.0


In [6]:
# Generate a unique client ID for each row
df['client_id'] = [str(uuid.uuid4()) for _ in range(len(df))]

# Set the index of the dataframe to the client_id column
df.set_index('client_id', inplace=True)

In [7]:
# Print the dataframe
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
client_id,,,,,,,,,,,,,,,
fc6fa183-4ef0-4a87-a24e-59652267a46b,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
64722f3d-d4c8-4f0a-a01b-6f7140872309,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
760ba3b2-2352-4d87-9dd4-a7d19fbdecf6,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
d8952dce-2aba-4e93-ab44-ebf17ee78a87,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
5133abcc-9228-4303-842d-39099a2c4ede,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1d4d47a0-648c-4a1d-913e-fac37e3af295,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
ff75f43a-6eb7-457c-bf1c-e7ec6e6cdf7b,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
aba3f06f-2fcb-4722-8a63-fed051d50501,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0


In [8]:
# Get the columns with NaN values
nan_cols = df.columns[df.isnull().any()]

# Print the columns with NaN values as a list
print(nan_cols.tolist())

['GramComplex']


In [9]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
# Fill missing values in the numeric columns with the mean value
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [10]:
# Replace the 'NoDx' value with 0 and the 'Dx' value with 1
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].replace("NoDx", 0).replace("Dx", 1)
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].astype(int)

In [11]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
print(len(train_data), len(test_data), type(train_data))

699 175 <class 'pandas.core.frame.DataFrame'>


In [13]:
print(test_data.dtypes)

Level.gender             int64
Level.ethnic             int64
Level.med                int64
Level.fed                int64
Level.income             int64
FamHist                  int64
Percentile             float64
GCC                    float64
Degree                 float64
HC                     float64
Between                float64
PathLength             float64
CombWords              float64
AnyLangorReadDxOnly      int64
GramComplex            float64
dtype: object


In [21]:
# ----------------------------------------------------Creating Federated Data--------------------------------------------------

#Get a list of client_names
client_names = df.index.values

# Get a numpy array of each client's values for all collumns, except the label column
client_data = df[["Level.gender", "Level.ethnic", "Level.med", "Level.fed", "Level.income", "FamHist",\
                 "Percentile", "GCC", "Degree", "HC", "Between", "PathLength", "CombWords", "GramComplex"]].values

# Get a numpy array of each client's value for only the label column
client_labels = df["AnyLangorReadDxOnly"].values

# Get an array of a tuple(client_data element, respective client_label)
data = list(zip(client_data, client_labels))

#shard data and place at each client
shards = [data[i:i + 1] for i in range(0, len(data), 1)]

# Assert number of clients must equal number of shards
assert(len(shards) == len(client_names))

# Create a dictionary with the client names as keys and the data columns as the values
clients = {client_names[i] : shards[i] for i in range(len(client_names))}
clients = {k: v[0] for k, v in clients.items()}


# Batch the data_shard into a an array of tensorflow dataset
def batch_data(data_shard, bs=32, pref=1):
    
    # Create a dataset from the labels and features of this client
    dataset = tf.data.Dataset.from_tensor_slices(collections.OrderedDict(
        x=tf.constant([data_shard[0]], dtype=tf.float64),
        y=tf.constant([[data_shard[1]]], dtype=tf.int64)
    ))
    
    # Shuffle and batch the dataset.
    return dataset.shuffle(32, seed=1).batch(bs)

# Run the function for each row in the clients dictionary, and add it back to a dictionary clients_batched
federated_data = []
for (client_name, data) in clients.items():
    federated_data.append(batch_data(data))


print(federated_data)

[<_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(Non

In [16]:
# -----------------------------------------------Create a Federated Model-------------------------------------------------------

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# This function creates a Sequential keras model
def create_compiled_keras_model():

    model = Sequential([
        Dense(64, activation='relu', input_shape=(14,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    return model

# Summarize the model (should be the same as the centralized model)
m = create_compiled_keras_model()
m.summary() 


# Shuffle the data randomly
random.seed(42)
random.shuffle(federated_data)

# Calculate the number of elements to select when splitting train and test data into a 80/20 split
n = int(len(federated_data) * 0.2)

# Use random.sample to randomly select n elements from the list
federated_test_data = federated_data[:n] # Represents central data that was collected with consent (without FL)
federated_train_data = federated_data[n:] # Represents FL client data

# This function creates a Sequential keras model and wraps it in a TFF model.
def model_fn():
  keras_model = create_compiled_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec, # Set the dataset's specifications
      loss=tf.keras.losses.BinaryCrossentropy(), # Set the loss function
      metrics=[tf.keras.metrics.BinaryAccuracy()]) # Set the metric function to measure accuracy

# Build the training process with the TFF model (as a function), the client-side optimizer and server-side optimizer
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate= 0.001),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate= 0.001))

# Take 10 samples of the training process and store the results in a file samples.txt
with open('samples.txt', 'w') as f:
    NUM_SAMPLES = 10
    for sample_num in range(NUM_SAMPLES):
        # For our implementation, it would be a challenge to pick randomly selected datasets, to simulate real world interactions.
        train_state = training_process.initialize()
        
        # Number of Federated learning computations to run
        NUM_ROUNDS = 10
        
        # Run 10 FL computations with the client data.
        for round_num in range(NUM_ROUNDS):
          result = training_process.next(train_state, federated_train_data)
          train_state = result.state
          train_metrics = result.metrics
          print('round {:2d}, metrics={}'.format(round_num, train_metrics), file=f)

        print("-----------------------------------End----------------------------------", file=f)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                960       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,073
Trainable params: 3,073
Non-trainable params: 0
_________________________________________________________________


2023-04-28 20:59:59.853814: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 20:59:59.854419: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-28 20:59:59.894929: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 20:59:59.895181: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-28 21:00:00.407641: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 21:00:00.408055: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-28 21:00:00.529404: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 21:00:00.529753: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


2023-04-28 21:00:05.951648: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:05.974523: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:05.975760: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:05.977356: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:05.980463: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:05.984511: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:06.111162: W external/org_tensorflow/tens

2023-04-28 21:00:09.184662: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:09.199279: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:09.200452: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:09.242914: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:09.509581: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:09.537168: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:09.788659: W external/org_tensorflow/tens

2023-04-28 21:00:12.500771: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:12.503361: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:12.507093: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:12.521275: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:12.522995: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:12.538892: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:00:12.701864: W external/org_tensorflow/tens

...

2023-04-28 21:05:29.901804: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:29.903064: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:29.906534: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:29.908766: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:30.047387: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:30.071892: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:30.140427: W external/org_tensorflow/tens

2023-04-28 21:05:32.216383: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:32.260721: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:32.330342: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:32.332336: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:32.444808: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:32.451121: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:32.477355: W external/org_tensorflow/tens

2023-04-28 21:05:35.440094: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:35.600918: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:35.656472: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:35.665934: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:35.679446: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:35.784747: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:35.867085: W external/org_tensorflow/tens

2023-04-28 21:05:39.676368: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:40.449211: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:40.493654: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:40.517715: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:40.519292: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:41.240510: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:41.247956: W external/org_tensorflow/tens

2023-04-28 21:05:44.439896: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:44.625016: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:44.697716: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:45.002508: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:45.189877: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:45.190361: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:45.196372: W external/org_tensorflow/tens

In [18]:
# Build an evaluation process
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)

# Initialize the evaluation process
evaluation_state = evaluation_process.initialize()

# Get the model weights from the final state of the training process
model_weights = training_process.get_model_weights(train_state)

# Set the model weights for the evaluation process
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

# Run an evaluation process, using the federated test data.
# This process happens on the server-side.
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

print(str(evaluation_output.metrics))

2023-04-28 21:05:48.332336: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 21:05:48.332699: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-28 21:05:48.347782: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 21:05:48.348053: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-28 21:05:48.376520: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 21:05:48.376827: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-28 21:05:48.400837: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-28 21:05:48.401174: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('binary_accuracy', 0.9310345), ('loss', 0.99576825), ('num_examples', 174), ('num_batches', 174)])), ('total_rounds_metrics', OrderedDict([('binary_accuracy', 0.9310345), ('loss', 0.99576825), ('num_examples', 174), ('num_batches', 174)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])


In [19]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)
print(str(evaluation_output.metrics))

2023-04-28 21:05:49.722316: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-28 21:05:49.837551: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled


OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('binary_accuracy', 0.9310345), ('loss', 0.99576825), ('num_examples', 174), ('num_batches', 174)])), ('total_rounds_metrics', OrderedDict([('binary_accuracy', 0.9310345), ('loss', 0.99576825), ('num_examples', 174), ('num_batches', 174)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])


2023-04-28 21:05:50.007653: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled


In [ ]:
# Reference: 
#TFF model Evaluation Process: https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification#evaluation
